# Produce Simple Star Subtracted Images

This script implements a simpler procedure to produce star subtracted images from the "swarped" images for a tile.

It produces subtracted images using the narrow band filter and the R band filter.

The subtractions invoving the N708 filter are just simple subtractions since nominally our swarped images have been
adjusted so that the stars are equally bright in all images.

The subtraction involving the R band filter is more complicated.  The steps involves removing (to first order) the SII and Ha
contaimination to the R band filter, to produce a R band image with only stars, and then subtracting the pure R band image
from the SII and H images.  

The naming conventions of the various images that are produced should be self explanatory

* SII_sub_N708, and Ha_sub_N708 is the images created using N708
* SII_sub_R, and Ha_sub_R are the images produced with the R_band
* r_sub is the R band images that contains only stars
* ha_s2_sub is (or is proportional to) the sum of the Ha and SII emission that was in the R band image

There are couple of things to note

* The algorithing to do the emission line subtraction from the R band emission can be improved by taking into account the
differend fileter widths of the Ha and NII fileters
* The routine at the bottom for processing individual fields is not what we want tin the end.  It's a kluge while we 
are developing a better subtraction algorithm.  It would be better to create the subtracted images immediately after the
tiles are swarped.



In [30]:
import matplotlib as plt
import numpy as np

from astropy.io import fits
from glob import glob

from astropy.table import Table

from astropy.stats import sigma_clipped_stats
from astropy.wcs import WCS

In [15]:
glob('../workspace/LMC_c42_d/7/L*fits')

['../workspace/LMC_c42_d/7/LMC_c42_7.R.t060.weight.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.N708.t400.weight.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.r_sub.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.s2_sub_r.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.R.t008.t060.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.N708.t060.t400.weight.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.Ha.t030.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t060.t800.weight.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.ha_s2_sub.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t060.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.R.t008.t060.weight.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.Ha.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.N708.t400.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t800.weight.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.R.t060.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.ha_sub_r.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7

In [16]:
test_ha='../workspace/LMC_c42_d/7/LMC_c42_7.Ha.t030.t800.fits'
test_s2='../workspace/LMC_c42_d/7/LMC_c42_7.SII.t060.t800.fits'
test_r='../workspace/LMC_c42_d/7/LMC_c42_7.R.t008.t060.fits'
test_n708='../workspace/LMC_c42_d/7/LMC_c42_7.N708.t060.t400.fits'

In [17]:
def fits_deep_copy(orig):
    '''
    Make a deep copy of a fits image
    
    Astropy does not have a way, apparently to make 
    a deep copy of an image; instead it makes shallow
    copies, which means that often one does not know
    what one is dealing with. 
    
    The basic problem here is that if one does not
    make deep copies, one will not necessary know
    what image you are working with, one that is
    pristine or one that has been modified
    
    This little routine
    avoids this issue.
    
    
    '''
    xhead=orig[0].header.copy()
    xdata=orig[0].data.copy()
    hdu=fits.PrimaryHDU(xdata)
    x=fits.HDUList([hdu])
    x[0].header=xhead
    return x

In [18]:
def make_rband_subtractions(ha,s2,r,outroot=''):
    
    ha_exists=False
    s2_exists=False
    r_exists=False

    
    try:
        zha=fits.open(ha)
        ha_exists=True
    except:
        print('The   Ha file could not be openened: %s' % ha)
    
    try:
        zs2=fits.open(s2)
        s2_exists=True
    except:
        print('The  SII file could not be openened: %s' % s2)
         
    try:
        zr=fits.open(r)
        r_exists=True
    except:
        print('The    R file could not be openened: %s' % r)    
        
    if ha_exists==False or s2_exists==False or r_exists==False:
        return
    
    if outroot=='':
        i=ha.rindex('/')
        xdir=ha[:i]
        xfile=ha[i+1:]

        word=xfile.split('.')
        outroot=word[0]
        outroot='%s/%s' % (xdir,word[0])
        # Use the directory path/first_word as the root, which is the usual case

    narrow=fits_deep_copy(zha)
    narrow[0].data=zha[0].data+zs2[0].data-2*zr[0].data
    narrow.writeto(outroot+'.ha_s2_sub.fits',overwrite=True)

    r_pure=fits_deep_copy(zr)
    r_pure[0].data-= (130/1500)* narrow[0].data
    r_pure.writeto(outroot+'.r_sub.fits',overwrite=True)    
    
    x=fits_deep_copy(zha)
    x[0].data-=r_pure[0].data
    x.writeto(outroot+'.ha_sub_r.fits',overwrite=True)        

    x=fits_deep_copy(zs2)
    x[0].data-=r_pure[0].data
    x.writeto(outroot+'.s2_sub_r.fits',overwrite=True)      

    return

make_rband_subtractions(test_ha,test_s2,test_r)

In [19]:
def make_n708_subtractions(ha,s2,n708,outroot=''):
    '''
    For narrow band subtraction we assume there is no emission line
    contamination, and since everything is scaled to the same level
    we just do a straight subtractin
    '''
    ha_exists=False
    s2_exists=False

    n708_exists=False
    
    try:
        zha=fits.open(ha)
        ha_exists=True
    except:
        print('The   Ha file could not be openened: %s' % ha)
    
    try:
        zs2=fits.open(s2)
        s2_exists=True
    except:
        print('The  SII file could not be openened: %s' % s2)
      
      
    try:
        zn708=fits.open(n708)
        n708_exists=True
    except:
        print('The N708 file could not be openened: %s' % n708)
        return
    
    if outroot=='':
        i=ha.rindex('/')
        xdir=ha[:i]
        xfile=ha[i+1:]

        word=xfile.split('.')
        outroot=word[0]
        outroot='%s/%s' % (xdir,word[0])
        
    if ha_exists and n708_exists:
        zha[0].data-=zn708[0].data
        zha.writeto(outroot+'.ha_sub_n708.fits',overwrite=True)

    if s2_exists and n708_exists:
        zs2[0].data-=zn708[0].data
        zs2.writeto(outroot+'.s2_sub_n708.fits',overwrite=True)
        
make_n708_subtractions(test_ha,test_s2,test_n708)    

In [52]:
def get_files(field='LMC_c42',tile=7,option='all'):
    '''
    Locate files for creating subtractions in a cell
    
    This routine is intended to be run from the directory where 
    the production routines are run, and is something of a kluge
    
    The options are:
    
        all - get the files with multiple expsures combined
        long - get the files with the a single but the longest exposres
        short - get the files with a single and the shorted exposures
        
    If short and long were not constructed separately with swarp, one
    will bet the same result.
    
    '''
    
    files=glob('../workspace/%s_d/%d/%s_%d.*fits' % (field,tile,field,tile))
    # print(files)
    
    # eliminated the weight files
    
    xfiles=[]
    for one in files:
        if one.count('weight')==0 and one.count('sub')==0:
            xfiles.append(one)
    # print(xfiles)
    
    times=[]
    ntimes=[]
    xfilt=[]
    for one in xfiles:
        words=one.split('/')
        
        barefile=words[-1]
        words=barefile.split('.')
        t=0
        nt=0
        xfilt.append(words[1])
        for one in words:
            if one[0]=='t':
                xt=int(one[1:])
                nt+=1
                #print(xt)
                t+=xt
        times.append(t)
        ntimes.append(nt)
    # print(len(times),times)
    # print(len(xfilt),xfilt)
    
    xtab=Table([xfiles,xfilt,ntimes,times],names=['Filename','Filter','Ntimes','SumT'])
    
    if option!='all':
        xtab=xtab[xtab['Ntimes']==1]
    
    # print(np.unique(xtab['Filter']))
    
    ha=xtab[xtab['Filter']=='Ha']
    s2=xtab[xtab['Filter']=='SII']
    r=xtab[xtab['Filter']=='R']
    n708=xtab[xtab['Filter']=='N708']
    
    # print(len(ha),len(s2),len(r),len(n708))
    
    if option=='all' or option == 'long':
        ha.sort('SumT')
        xha=ha[-1]['Filename']
        # print(xha)
        
        s2.sort('SumT')
        xs2=s2[-1]['Filename']
        
        r.sort('SumT')
        xr=r[-1]['Filename']
        
        
        n708.sort('SumT')
        xn708=n708[-1]['Filename']
        
    elif option=='short':
        ha.sort('SumT')
        xha=ha[0]['Filename']

        
        s2.sort('SumT')
        xs2=s2[-1]['Filename']
        
        r.sort('SumT')
        xr=r[-1]['Filename']
        

        
        n708.sort('SumT')
        xn708=n708[-1]['Filename']   
        
    else:
        print('Error: Unkown option')
        return
        
    
    return xha,xs2,xr,xn708
    
    
        
        
    
get_files()

('../workspace/LMC_c42_d/7/LMC_c42_7.Ha.t030.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t060.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.R.t008.t060.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.N708.t060.t400.fits')

In [53]:
get_files(option='short')

('../workspace/LMC_c42_d/7/LMC_c42_7.Ha.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.R.t060.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.N708.t400.fits')

In [54]:
get_files(option='long')

('../workspace/LMC_c42_d/7/LMC_c42_7.Ha.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.SII.t800.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.R.t060.fits',
 '../workspace/LMC_c42_d/7/LMC_c42_7.N708.t400.fits')

In [58]:
def do_one_tile(field='LMC_c42',tile=7,option='all',root=''):
    ha,s2,r,n708=get_files(field,tile,option)
    make_rband_subtractions(ha,s2,r,root)
    make_n708_subtractions(ha,s2,n708,root)
do_one_tile()

In [59]:
def do_one_field(field='LMC_c42',option='all',root=''):
    i=1
    while i<17:
        do_one_tile(field,i,option,root)
        i+=1

do_one_field()

In [60]:
do_one_field(field='LMC_c45',option='all',root='')